## Assignment 02 – Cryptocurrency Portfolio

Group 3: Shufeng Li, Wenlin Ye, Yifei Yin  
Github link: https: // github.com/yyin422/fall23-5700-group-3/tree/main/HW2


#### 1 Importing data and packages


In [1]:
import pandas_datareader.data as pdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
import time
from urllib.parse import urlencode
from typing import Dict, List, Union, Optional, Any

plt.style.use('bmh')

#### 2 Gather Coin token ID information from Coinmarket, Binance, or Coinbase.
You will need to search on any of the three websites to get the token IDs. The video shows how to do it through Binance and coin market cap. A slight difference between the Binance token and the coin market token is that the Binance token adds BUSD or USDT at the end, while the CoinMarket token does not. For example, Bitcoin is represented as BTC in CoinMarket and BTCBUSD or BTCUSDT in Binance. This represents the currency conversion to a stablecoin instead of a fiat currency.

For the sake of this bit, you can choose any 30 cryptocurrencies except Bitcoin and Ethereum. Here are the categories to select from

High market cap group - this will include the most widely traded coins. We will use the top 5 coins, namely, Bitcoin (BTC-USD), Ethereum (ETH-USD), Litecoin (LTC-USD), Teather (USDT), and BNB (BNB-USD).
Mid-market cap group will include coins with a market cap between 1 billion and 20 billion. We will use the top 5 coins, namely, Cardano (ADA-USD), Polygon (MATIC-USD), Dogecoin (DOGE-USD), Ripple (XRP-USD), and Binance USD (BUSD).
The low market cap group will include coins with a market cap between 100 million and 1 billion. We will use the top 20 coins.
Make sure to list the ticker ids in a list of strings

#### 2.1 Getting Data From Binance
Create a function that takes ticker interval, start_time, and end_time as input. We want to get all the data for the selected tickers and save them in data chosen frames. There is a way to automate this by using a function as well.

For each coin/token, you will get a dataframe of daily movements from start to end. The date and time are set as the index. Close, Symbol, and index are the most critical columns, and we want to subset them in a" coins" dataframe for all the coins.
Rename close to "Coinname_closeprice." Replace the coin name with the name of the coin and token id.

In [2]:
from Binance import Binance
import pandas as pd
from pandas import json_normalize

ModuleNotFoundError: No module named 'Binance'

In [4]:
def binance_candle_stick(ticker: str, interval: str = '1d', start_time: Optional[int] = None, end_time: Optional[int] = None) -> pd.DataFrame:
    """Get historical data from Binance

    Args:
        ticker (str): The ticker symbol, e.g. BTCUSDT
        interval (str, optional): The price interval, '1d' ->'daily'. Defaults to '1d'.
        start (Optional[int], optional): Starting date, should be long int format. Defaults to None.

    Returns:
        pd.DataFrame: A dataframe with historical data
    """

    if start_time is None:
        start_time = int(datetime.datetime(2020, 6, 1).timestamp()*1000)
    if end_time is None:
        end_time = int(datetime.datetime.now().timestamp()*1000)

    payload = {
        'symbol': ticker,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000
    }
    query_string = urlencode(payload, True)
    url = 'https://api.binance.us/api/v3/klines'
    if query_string:
        url = url + '?' + query_string
    response = requests.get(url)
    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']
    data = pd.DataFrame(response.json(), columns=columns, dtype=float)
    data['open_time'] = pd.to_datetime(data['open_time'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
    data.set_index('open_time', inplace=True)

    return data[['open', 'high', 'low', 'close', 'volume', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol']]

NameError: name 'Optional' is not defined

In [5]:
def obtain_binance_exchangeInfo() -> pd.DataFrame:
    """Get the exchange info from Binance"""
    url = 'https://api.binance.us/api/v3/exchangeInfo'
    response = requests.get(url)
    data = pd.json_normalize(response.json()['symbols'])

    return data

In [6]:
exchangeInfo = obtain_binance_exchangeInfo()
asset = exchangeInfo.baseAsset.unique()[2:40]

In [7]:
symbol = exchangeInfo[exchangeInfo.baseAsset.isin(asset)][exchangeInfo.quoteAsset == 'USD'].symbol.values[:30]
# symbol

/var/folders/s1/2z11zjf503j5424zkxq225nw0000gn/T/ipykernel_83353/4175076379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  symbol = exchangeInfo[exchangeInfo.baseAsset.isin(asset)][exchangeInfo.quoteAsset == 'USD'].symbol.values[:30]


In [8]:

def clean_single(data:pd.DataFrame, symbol:str) -> pd.DataFrame:
    """Clean the data

    Args:
        data (pd.DataFrame): Original data
        symbol (str): The ticker symbol

    Returns:
        pd.DataFrame: Return a cleaned dataframe with only close price and symbol as column name.
    """
    data = data['close']
    data = data.rename(symbol)
    return data.reset_index()

In [13]:
# Obtain all data of the symbols
all_data = {}
for smb in symbol:
    all_data[smb] = binance_candle_stick(smb, interval = '1d')
    time.sleep(0.1)

In [17]:
all_data['BCHUSD'].head()

,open,high,low,close,volume,qav,num_trades,taker_base_vol,taker_quote_vol
open_time,,,,,,,,,
2020-06-02 00:00:00,253.44,269.25,242.01,251.17,1758.48550,449529.2852,2325.0,992.11013,253801.7295
2020-06-03 00:00:00,250.76,253.80,246.60,253.18,635.46949,159400.3513,624.0,345.38646,86712.3188
2020-06-04 00:00:00,253.39,260.69,247.80,256.50,881.60028,225372.0124,874.0,477.01974,121609.4981
2020-06-05 00:00:00,256.43,263.04,254.53,255.90,678.38959,174886.7101,671.0,242.19525,62602.9948
2020-06-06 00:00:00,255.43,257.82,251.20,253.28,528.49909,134208.6292,462.0,294.56256,74799.2667


#### 2.2 Combining the dataframe
The updated dataframe should have the index and close price of all the coins of interest. We will use this to calculate the efficient frontier for the portfolio.

In [15]:
coins = pd.DataFrame()
for key, value in all_data.items():
    # Ensure 'open_time' is a column and not an index in value
    value = value.reset_index()
    
    # If coins DataFrame is empty, simply assign the current DataFrame value to coins
    if coins.empty:
        coins = value
        continue
    
    # Merge coins with the current DataFrame value based on 'open_time'
    coins = pd.merge(coins, value, on='open_time', how='inner')

# Convert the 'open_time' column to datetime and set it as index only after all merges are done
coins['open_time'] = pd.to_datetime(coins['open_time'], format='%Y-%m-%d %H:%M:%S')
coins.set_index('open_time', inplace=True)



/var/folders/s1/2z11zjf503j5424zkxq225nw0000gn/T/ipykernel_83353/3735920777.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'taker_base_vol_x', 'close_x', 'open_x', 'volume_x', 'qav_x', 'high_x', 'taker_quote_vol_x', 'low_x', 'num_trades_x'} in the result is deprecated and will raise a MergeError in a future version.
  coins = pd.merge(coins, value, on='open_time', how='inner')
/var/folders/s1/2z11zjf503j5424zkxq225nw0000gn/T/ipykernel_83353/3735920777.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'taker_base_vol_x', 'close_x', 'open_x', 'volume_x', 'qav_x', 'high_x', 'taker_quote_vol_x', 'low_x', 'num_trades_x'} in the result is deprecated and will raise a MergeError in a future version.
  coins = pd.merge(coins, value, on='open_time', how='inner')
/var/folders/s1/2z11zjf503j5424zkxq225nw0000gn/T/ipykernel_83353/3735920777.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'taker_base_vol_x', 'close_x', 'open_x', '

In [16]:
coins.head()

,open_x,high_x,low_x,close_x,volume_x,qav_x,num_trades_x,taker_base_vol_x,taker_quote_vol_x,open_y,...,taker_quote_vol_x,open_y,high_y,low_y,close_y,volume_y,qav_y,num_trades_y,taker_base_vol_y,taker_quote_vol_y
open_time,,,,,,,,,,,,,,,,,,,,,
2020-06-23,242.25,244.28,239.38,240.49,183.29164,44321.8087,322.0,92.03304,22256.9924,44.37,...,39872.1429,2.5726,2.5899,2.5683,2.5683,1024.18,2642.9488,33.0,789.47,2037.7954
2020-06-24,240.16,245.09,230.00,232.51,416.29881,98497.0753,629.0,166.32068,39466.1529,44.19,...,25486.0215,2.5715,2.5973,2.4800,2.4868,4450.94,11362.6596,114.0,3097.00,7925.5204
2020-06-25,232.52,234.57,226.19,232.33,198.95030,46178.2788,276.0,53.59672,12436.9729,42.49,...,42762.5933,2.5137,2.5137,2.4671,2.4801,433.94,1081.1306,20.0,305.17,760.4697
2020-06-26,230.95,233.17,228.12,230.82,188.70994,43602.7461,397.0,69.51320,16019.9054,42.24,...,7287.9616,2.4750,2.5056,2.4201,2.4867,2653.39,6533.9193,84.0,1287.73,3160.4356
2020-06-27,229.75,231.22,200.01,216.05,953.67012,204255.4208,720.0,409.01277,87491.5878,42.93,...,58517.1474,2.4867,2.4868,2.2795,2.3365,4009.03,9394.9744,145.0,2664.61,6195.0610


#### 3. Calculating the Efficient Frontier
To calculate an efficient frontier, we will need to calculate each coin's means, standard deviations, and covariances.

Calculate the mean of the dataframe generated in 2.2 using pd.df.mean() method
Calculate the covariance matrix of the dataframe generated in 2.2 using pd.df.cov() method
Use the function in the video to generate a complete crypto portfolio for $10,000.

In [ ]:
def calc_daily_returns(df):
    return (df.pct_change())
 #calculate the daily percent change between the 
 
coin_daily_returns = calc_daily_returns(Combined data)
coin_daily_returns.head()


In [ ]:
def calc_mean_daily_returns(daily_returns):
    return(daily_returns.mean())
# calculate the mean daily returns for each coin
coin_mean_daily_returns = calc_mean_daily_returns(coin_daily_returns)
coin_mean_daily_returns

In [ ]:
def calc_covariance_matrix(daily_returns):
    return (daily_returns.cov())
crypto_cov_matrix = calc_covariance_matrix(coin_daily_returns)
print(crypto_cov_matrix)

In [ ]:
def portfolio_statistics(coins, number_portfolios, mean daily_returns, cov matrix):
    len_coins=len(coins)
    results_temp = np.zeros((4 + len coins - 1, number_portfolios))
    for i in range(number_portfolios):# select random weights for portfolio holdings
    weights = np.array(np.random.random(len_coins))

# rebalance weights to sum to 1
    weights /= np .sum(weights)
# calculate portfolio return and volatility
    portfolio_return = np.sum(mean_daily_returns * weights) * 252
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    
# 3month T-Bill yield used as risk free rate of return
    risk_free_return = 0.0139
# store results in results arrayresults 
    results_temp[0, i]= portfolio_return
    results_temp[1, i] = portfolio_std_dev
# store sharpe Ratio (returnvolatilitynisk free rate element excluded forsimplicity
    results_temp[2, i] = (results_temp[0, i] - risk_free_return) / results_temp[1, i]
# iterate through the weight vector and add data to results array
    for j in range(len(weights)):
        results_temp[j + 3, i] = weights[j]
# convert results array to Pandas DataFramesharpe'
    results_df = pd.DataFrame(results_temp.T, columns=['returns', 'stdev', 'sharpe', coins[0],
                                                       coins[1],coins[2]])
return (results_df) 

In [ ]:
crypto_results = portfolio_statistics(coins,10000, coin_mean_daily_returns, crypto_cov_matrix)
crypto_results .head(15)

#### 4. Locate the Best portfolios
The percentages returned from the best portfolio search represent the total assets added to each coin from the $10, 000.

#### 4.1 Locate the position of the portfolio with the highest Sharpe Ratio

In [ ]:
def max_sharpe_ratio(results_df):
    """locate portfolio with highest Sharpe Ratio"""
    return results_df.iloc[results_df['sharpe' ].idxmax()]
crypto_max_sharpe_portfolio = maxs_sharpe_ratio(cryptoresults)
crypto_max_sharpe_portfolio

#### 4.2 Locate the portfolio with the lowest volatility


In [ ]:
def min_volatility(results_df):
#"""locate portfolio with lowest volatility"""
    return results_df.iloc[results_df['stdev'].idxmin()]
crypto_min_vol_portfolio = min_volatility(cryptoresults)
print(crypto_min_vol_portfolio)

#### 4.3 Plot the efficient portfolio

In [ ]:
def plot_graph(results_df, max_sharpe_port, min_vol_port):
    ax = results_df.plot(kind= 'scatter', x = 'stdev', y='returns', s = 30,
                         c=results_df.sharpe, cmap='RdYlBu',edgecolors='.1')
ax.grid(False, color='#777777", linestyle='-.', linewidth=0.5)
ax.set_facecolor('#f7f7f7')
ax.set_xlabel('Volatility')
ax.set_ylabel(Returns')
ax.tick_params(labelsize = 14)
# # plot red star to highlight the position of the portfolio with highest Sharpe Ratio
ax.scatter(max_sharpe_port[1], max_sharpe_port[0], marker=(5, 1, 0), color='#993300'. s= 200)
# # plot green star to highlight position of minimum variance portfolio
ax.scatter(min_vol_port[1], min_vol_port[0], marker=(5, 1, 0), color='#009933', s = 200)
plot_graph(crypto_results, crypto_max_sharpe_portfolio, crypto_min_vol_portfolio)
plt.show()